In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import random
from faker import Faker
import numpy as np
from random import randint
import time

In [ ]:
'''
simple_ratio on 10k:
- fuzzywuzzy: 16min
- rapidfuzz: 4min
- optimized rapidfuzz: 3sec
- optimized & parallelized rapidfuzz: nerulat

simple_ratio on 100k:
- fuzzywuzzy: - (nerulat, poate dura ore intregi din cauza search-ului)
- rapidfuzz: - (nerulat, posibil 1-2-3h)
- optimized rapidfuzz: 13min
- optimized & parallelized rapidfuzz: 6min
'''

## Fuzzy Matching with NaN vals dropped ##

In [68]:
df1 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df10000.csv")
df2 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df10000.csv")

# Drop rows with NaN values only for specific columns in both dataframes
columns_to_check = ['FirstName', 'LastName']
# df1_cleaned = df1.dropna(subset=columns_to_check)
df2 = df2.dropna(subset=columns_to_check)

print("After dropping rows with NaNs in 'FirstName' and 'LastName':")
# print("df1_cleaned:")
# print(df1_cleaned.isnull().sum())

print("df2 cleaned:")
print(df2.isnull().sum())
print(df2.shape)

After dropping rows with NaNs in 'FirstName' and 'LastName':
df2 cleaned:
Unnamed: 0       0
FirstName        0
LastName         0
DateOfBirth    851
dtype: int64
(8137, 4)


In [69]:
hr_df = df1.copy()
client_df = df2.copy()
client_df['Revenue'] = [randint(2000, 500000) for _ in range(df2.shape[0])]
print(f"DF Shapes: hr_df = {hr_df.shape}, client_df = {client_df.shape}")

DF Shapes: hr_df = (10000, 4), client_df = (8137, 5)


In [70]:
hr_df['FirstName'] = hr_df['FirstName'].str.lower()
hr_df['LastName'] = hr_df['LastName'].str.lower()
hr_df['FullName'] = hr_df['FirstName']+hr_df['LastName']
hr_df.head(10)


,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName
0,0,timothy,lynch,1981-05-17,timothylynch
1,1,dalton,rogers,1988-01-10,daltonrogers
2,2,kathleen,cooley,1945-10-08,kathleencooley
3,3,kevin,jones,1946-08-21,kevinjones
4,4,christine,martin,1951-03-23,christinemartin
5,5,robert,walker,1961-02-13,robertwalker
6,6,jeffrey,rios,2005-10-11,jeffreyrios
7,7,robert,brown,1983-03-24,robertbrown
8,8,wendy,richardson,2014-07-09,wendyrichardson
9,9,melissa,barber,1996-03-17,melissabarber


In [71]:
client_df['FirstName'] = client_df['FirstName'].str.lower()
client_df['LastName'] = client_df['LastName'].str.lower()
client_df['FullName'] = client_df['FirstName']+client_df['LastName']
client_df.head(10)

,Unnamed: 0,FirstName,LastName,DateOfBirth,Revenue,FullName
0,0,timothy,lynch,1981-05-17,337151,timothylynch
2,2,kathleen,cooley,1945-10-08,333395,kathleencooley
3,3,kevin,joens,NaN,269431,kevinjoens
4,4,chrisitne,martin,1951-03-23,55473,chrisitnemartin
5,5,orbert,walker,1961-02-13,476512,orbertwalker
7,7,robert,brown,1983-03-24,16422,robertbrown
8,8,wenyd,richardson,2014-07-09,371244,wenydrichardson
9,9,melissa,barber,1996-03-17,475648,melissabarber
10,10,kimberly,dnucan,2012-01-27,226340,kimberlydnucan
12,12,robetr,cochran,2004-01-02,80122,robetrcochran


In [72]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function to match two columns using fuzzywuzzy and return both match and score
# def fuzzy_match_with_score(row, choices, scorer=fuzz.token_sort_ratio, threshold=80):
#     match = process.extractOne(row, choices, scorer=scorer)
#     if match and match[1] >= threshold:
#         return match[0], match[1]
#     return None, 0

def fuzzy_match_with_score(row, choices, scorer, threshold=80):
    choices_list = list(choices)  # Ensure choices is a list
    match = process.extractOne(row, choices_list, scorer=scorer)
    if match and match[1] >= threshold:
        matched_value, score = match
        match_index = choices_list.index(matched_value)
        return matched_value, score, match_index
    return None, 0, -1


def best_match_with_score(param_df, scorer) -> pd.DataFrame:
    # Apply fuzzy matching to df1
    df = param_df.copy()
    df['MatchedFullName'], df['FullNameScore'], df['FullNameIndex'] = zip(*df['FullName'].dropna().apply(lambda x: fuzzy_match_with_score(x, client_df_full_names, scorer=scorer)))
    df['MatchedFirstName'], df['FirstNameScore'], df['FirstNameIndex'] = zip(*df['FirstName'].dropna().apply(lambda x: fuzzy_match_with_score(x, client_df_first_names, scorer=scorer)))
    df['MatchedLastName'], df['LastNameScore'], df['LastNameIndex'] = zip(*df['LastName'].dropna().apply(lambda x: fuzzy_match_with_score(x, client_df_last_names, scorer=scorer)))

    # Fill NaN values for rows that were originally NaN
    df['MatchedFullName'].fillna('_', inplace=True)
    df['FullNameScore'].fillna(0, inplace=True)
    df['FullNameIndex'].fillna(-1, inplace=True)
    df['MatchedFirstName'].fillna('_', inplace=True)
    df['FirstNameScore'].fillna(0, inplace=True)
    df['FirstNameIndex'].fillna(-1, inplace=True)
    df['MatchedLastName'].fillna('_', inplace=True)
    df['LastNameScore'].fillna(0, inplace=True)
    df['LastNameIndex'].fillna(-1, inplace=True)

    return df

# Prepare lists of names/choices from df2 to match against
client_df_first_names = client_df['FirstName'].dropna().unique()
client_df_last_names = client_df['LastName'].dropna().unique()
client_df_full_names = client_df['FullName'].dropna().unique()

hr_df.head(100)

,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName
0,0,timothy,lynch,1981-05-17,timothylynch
1,1,dalton,rogers,1988-01-10,daltonrogers
2,2,kathleen,cooley,1945-10-08,kathleencooley
3,3,kevin,jones,1946-08-21,kevinjones
4,4,christine,martin,1951-03-23,christinemartin
...,...,...,...,...,...
95,95,jill,hancock,1934-04-04,jillhancock
96,96,christopher,medina,2002-09-15,christophermedina
97,97,john,jackson,1957-02-20,johnjackson
98,98,ashley,mendez,1979-11-27,ashleymendez


In [73]:
# SIMPLE RATIO METHOD
start_time = time.time()
df_simple_ratio = best_match_with_score(param_df = hr_df, scorer = fuzz.ratio)
print(f"Elapsed time simple ratio: {(time.time() - start_time):.2f} seconds")
df_simple_ratio.to_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\output_dataframes\fuzzy_df10k_simple_ratio.csv")
# Show results
df_simple_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time simple ratio: 970.41 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,timothy,timothy,100,0,lynch,lynch,100,0,timothylynch,100,0
1,dalton,dalton,100,562,rogers,rogers,100,408,_,0,-1
2,kathleen,kathleen,100,1,cooley,cooley,100,1,kathleencooley,100,1
3,kevin,kevin,100,2,jones,jones,100,93,kevinjones,100,4194
4,christine,christine,100,131,martin,martin,100,3,chrisitnemartin,93,3
...,...,...,...,...,...,...,...,...,...,...,...
95,jill,jill,100,72,hancock,hancock,100,729,_,0,-1
96,christopher,christopher,100,46,medina,medina,100,71,christophermedina,100,75
97,john,john,100,70,jackson,jackson,100,81,johnjackson,100,3994
98,ashley,ashley,100,71,mendez,mendez,100,342,ashleymendze,92,77


In [48]:
# PARTIAL RATIO METHOD
# Apply fuzzy matching to df1
start_time = time.time()
df_partial_ratio = best_match_with_score(param_df = hr_df, scorer = fuzz.partial_ratio)
print(f"Elapsed time partial ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_partial_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'LastName', 'MatchedLastName', 'LastNameScore', 'MatchedFullName','FullNameScore']].head(100)

Elapsed time partial ratio: 44.20 seconds


,FirstName,MatchedFirstName,FirstNameScore,LastName,MatchedLastName,LastNameScore,MatchedFullName,FullNameScore
0,sarah,sarah,100,mitchell,mitchell,100,sarahmitchell,100
1,rhonda,rhonda,100,villegas,le,100,rhondavilleags,93
2,ryan,ryan,100,jones,jones,100,ryanjones,100
3,chloe,chleo,80,gonzales,gonzales,100,chleogonzales,92
4,tyler,tyler,100,serrano,serrano,100,tylerserrano,100
...,...,...,...,...,...,...,...,...
95,joseph,joseph,100,leon,leon,100,jsoephleon,90
96,cynthia,cynthia,100,arroyo,arroyo,100,cynthiaarroyo,100
97,jessica,jessica,100,clark,clark,100,jessiacclark,92
98,robert,robert,100,ramirez,ramirez,100,robertramirez,100


In [49]:
# TOKEN_SORT_RATIO METHOD
# Apply fuzzy matching to df1
start_time = time.time()
df_token_sort_ratio = best_match_with_score(param_df = hr_df, scorer = fuzz.token_sort_ratio)
print(f"Elapsed time token sort ratio: {(time.time() - start_time):.2f} seconds")

# Show results
df_token_sort_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'LastName', 'MatchedLastName', 'LastNameScore', 'MatchedFullName','FullNameScore']].head(100)

Elapsed time token sort ratio: 22.80 seconds


,FirstName,MatchedFirstName,FirstNameScore,LastName,MatchedLastName,LastNameScore,MatchedFullName,FullNameScore
0,sarah,sarah,100,mitchell,mitchell,100,sarahmitchell,100
1,rhonda,rhonda,100,villegas,villeags,88,rhondavilleags,93
2,ryan,ryan,100,jones,jones,100,ryanjones,100
3,chloe,chleo,80,gonzales,gonzales,100,chleogonzales,92
4,tyler,tyler,100,serrano,serrano,100,tylerserrano,100
...,...,...,...,...,...,...,...,...
95,joseph,joseph,100,leon,leon,100,jsoephleon,90
96,cynthia,cynthia,100,arroyo,arroyo,100,cynthiaarroyo,100
97,jessica,jessica,100,clark,clark,100,jessiacclark,92
98,robert,robert,100,ramirez,ramirez,100,robertramirez,100


In [50]:
# TOKEN_SET_RATIO METHOD
# Apply fuzzy matching to df1
start_time = time.time()
df_token_set_ratio = best_match_with_score(param_df = hr_df, scorer = fuzz.token_set_ratio)
print(f"Elapsed time token set ratio: {(time.time() - start_time):.2f} seconds")

# Show results
df_token_set_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'LastName', 'MatchedLastName', 'LastNameScore', 'MatchedFullName','FullNameScore']].head(100)

Elapsed time token set ratio: 32.63 seconds


,FirstName,MatchedFirstName,FirstNameScore,LastName,MatchedLastName,LastNameScore,MatchedFullName,FullNameScore
0,sarah,sarah,100,mitchell,mitchell,100,sarahmitchell,100
1,rhonda,rhonda,100,villegas,villeags,88,rhondavilleags,93
2,ryan,ryan,100,jones,jones,100,ryanjones,100
3,chloe,chleo,80,gonzales,gonzales,100,chleogonzales,92
4,tyler,tyler,100,serrano,serrano,100,tylerserrano,100
...,...,...,...,...,...,...,...,...
95,joseph,joseph,100,leon,leon,100,jsoephleon,90
96,cynthia,cynthia,100,arroyo,arroyo,100,cynthiaarroyo,100
97,jessica,jessica,100,clark,clark,100,jessiacclark,92
98,robert,robert,100,ramirez,ramirez,100,robertramirez,100


## Rapid Fuzz Matching with NaN vals dropped ##

In [75]:
df1 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df100000.csv")
df2 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df100000.csv")

# Drop rows with NaN values only for specific columns in both dataframes
columns_to_check = ['FirstName', 'LastName']
# df1_cleaned = df1.dropna(subset=columns_to_check)
df2 = df2.dropna(subset=columns_to_check)

print("After dropping rows with NaNs in 'FirstName' and 'LastName':")
# print("df1_cleaned:")
# print(df1_cleaned.isnull().sum())

print("df2 cleaned:")
print(df2.isnull().sum())
print(df2.shape)

After dropping rows with NaNs in 'FirstName' and 'LastName':
df2 cleaned:
Unnamed: 0       0
FirstName        0
LastName         0
DateOfBirth    851
dtype: int64
(8137, 4)


In [76]:
hr_df = df1.copy()
client_df = df2.copy()
client_df['Revenue'] = [randint(2000, 500000) for _ in range(df2.shape[0])]
print(f"DF Shapes: hr_df = {hr_df.shape}, client_df = {client_df.shape}")

DF Shapes: hr_df = (10000, 4), client_df = (8137, 5)


In [77]:
hr_df['FirstName'] = hr_df['FirstName'].str.lower()
hr_df['LastName'] = hr_df['LastName'].str.lower()
hr_df['FullName'] = hr_df['FirstName']+hr_df['LastName']
hr_df.head(10)


,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName
0,0,timothy,lynch,1981-05-17,timothylynch
1,1,dalton,rogers,1988-01-10,daltonrogers
2,2,kathleen,cooley,1945-10-08,kathleencooley
3,3,kevin,jones,1946-08-21,kevinjones
4,4,christine,martin,1951-03-23,christinemartin
5,5,robert,walker,1961-02-13,robertwalker
6,6,jeffrey,rios,2005-10-11,jeffreyrios
7,7,robert,brown,1983-03-24,robertbrown
8,8,wendy,richardson,2014-07-09,wendyrichardson
9,9,melissa,barber,1996-03-17,melissabarber


In [78]:
client_df['FirstName'] = client_df['FirstName'].str.lower()
client_df['LastName'] = client_df['LastName'].str.lower()
client_df['FullName'] = client_df['FirstName']+client_df['LastName']
client_df.head(10)

,Unnamed: 0,FirstName,LastName,DateOfBirth,Revenue,FullName
0,0,timothy,lynch,1981-05-17,451402,timothylynch
2,2,kathleen,cooley,1945-10-08,111431,kathleencooley
3,3,kevin,joens,NaN,466036,kevinjoens
4,4,chrisitne,martin,1951-03-23,495813,chrisitnemartin
5,5,orbert,walker,1961-02-13,90249,orbertwalker
7,7,robert,brown,1983-03-24,401789,robertbrown
8,8,wenyd,richardson,2014-07-09,27300,wenydrichardson
9,9,melissa,barber,1996-03-17,387558,melissabarber
10,10,kimberly,dnucan,2012-01-27,248619,kimberlydnucan
12,12,robetr,cochran,2004-01-02,35467,robetrcochran


In [79]:
import pandas as pd
import rapidfuzz

def rapidfuzz_match_with_score(row, choices, scorer, threshold=80):
    choices_list = list(choices)  # Ensure choices is a list
    match = process.extractOne(row, choices_list, scorer=scorer)
    if match and match[1] >= threshold:
        matched_value, score, match_index = match[0], match[1], choices_list.index(match[0])
        return matched_value, score, match_index
    return None, 0, -1


def rapidfuzz_best_match_with_score(param_df, scorer) -> pd.DataFrame:
    # Apply fuzzy matching to df1
    df = param_df.copy()
    df['MatchedFullName'], df['FullNameScore'], df['FullNameIndex'] = zip(*df['FullName'].dropna().apply(lambda x: rapidfuzz_match_with_score(x, client_df_full_names, scorer=scorer)))
    df['MatchedFirstName'], df['FirstNameScore'], df['FirstNameIndex'] = zip(*df['FirstName'].dropna().apply(lambda x: rapidfuzz_match_with_score(x, client_df_first_names, scorer=scorer)))
    df['MatchedLastName'], df['LastNameScore'], df['LastNameIndex'] = zip(*df['LastName'].dropna().apply(lambda x: rapidfuzz_match_with_score(x, client_df_last_names, scorer=scorer)))

    # Fill NaN values for rows that were originally NaN
    df['MatchedFullName'].fillna('_', inplace=True)
    df['FullNameScore'].fillna(0, inplace=True)
    df['FullNameIndex'].fillna(-1, inplace=True)
    df['MatchedFirstName'].fillna('_', inplace=True)
    df['FirstNameScore'].fillna(0, inplace=True)
    df['FirstNameIndex'].fillna(-1, inplace=True)
    df['MatchedLastName'].fillna('_', inplace=True)
    df['LastNameScore'].fillna(0, inplace=True)
    df['LastNameIndex'].fillna(-1, inplace=True)

    return df

# Prepare lists of names/choices from df2 to match against
client_df_first_names = client_df['FirstName'].dropna().unique()
client_df_last_names = client_df['LastName'].dropna().unique()
client_df_full_names = client_df['FullName'].dropna().unique()

hr_df.head(100)

,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName
0,0,timothy,lynch,1981-05-17,timothylynch
1,1,dalton,rogers,1988-01-10,daltonrogers
2,2,kathleen,cooley,1945-10-08,kathleencooley
3,3,kevin,jones,1946-08-21,kevinjones
4,4,christine,martin,1951-03-23,christinemartin
...,...,...,...,...,...
95,95,jill,hancock,1934-04-04,jillhancock
96,96,christopher,medina,2002-09-15,christophermedina
97,97,john,jackson,1957-02-20,johnjackson
98,98,ashley,mendez,1979-11-27,ashleymendez


In [80]:
# SIMPLE RATIO METHOD
start_time = time.time()
df_simple_ratio = rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.ratio)
print(f"Elapsed time simple ratio: {(time.time() - start_time):.2f} seconds")
df_simple_ratio.to_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\output_dataframes\rapidfuzz_df10k_simple_ratio.csv")
# Show results
df_simple_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time simple ratio: 240.40 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,timothy,timothy,100.0,0,lynch,lynch,100.0,0,timothylynch,100.000000,0
1,dalton,dalton,100.0,562,rogers,rogers,100.0,408,_,0.000000,-1
2,kathleen,kathleen,100.0,1,cooley,cooley,100.0,1,kathleencooley,100.000000,1
3,kevin,kevin,100.0,2,jones,jones,100.0,93,kevinjones,100.000000,4194
4,christine,christine,100.0,131,martin,martin,100.0,3,chrisitnemartin,93.333333,3
...,...,...,...,...,...,...,...,...,...,...,...
95,jill,jill,100.0,72,hancock,hancock,100.0,729,_,0.000000,-1
96,christopher,christopher,100.0,46,medina,medina,100.0,71,christophermedina,100.000000,75
97,john,john,100.0,70,jackson,jackson,100.0,81,johnjackson,100.000000,3994
98,ashley,ashley,100.0,71,mendez,mendez,100.0,342,ashleymelendez,92.307692,5008


In [58]:
# PARTIAL RATIO METHOD
start_time = time.time()
df_partial_ratio = rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.partial_ratio)
print(f"Elapsed time partial ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_partial_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time partial ratio: 6.16 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.000000,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.000000,1,villegas,le,100.0,359,rhondavilleags,92.857143,1
2,ryan,ryan,100.000000,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,88.888889,3,gonzales,gonzales,100.0,3,chleogonzales,92.307692,3
4,tyler,tyler,100.000000,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.000000,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.000000,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.000000,28,clark,clark,100.0,72,jessiacclark,91.666667,75
98,robert,robert,100.000000,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


In [59]:
# TOKEN SORT RATIO METHOD
start_time = time.time()
df_token_sort_ratio = rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.token_sort_ratio)
print(f"Elapsed time token sort ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_token_sort_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time token sort ratio: 5.66 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.0,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.0,1,villegas,villeags,87.5,1,rhondavilleags,92.857143,1
2,ryan,ryan,100.0,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,80.0,3,gonzales,gonzales,100.0,3,chleogonzales,92.307692,3
4,tyler,tyler,100.0,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.0,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.0,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.0,28,clark,clark,100.0,72,jessiacclark,91.666667,75
98,robert,robert,100.0,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


In [60]:
# TOKEN SET RATIO METHOD
start_time = time.time()
df_token_set_ratio = rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.token_set_ratio)
print(f"Elapsed time token set ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_token_set_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time token set ratio: 7.37 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.0,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.0,1,villegas,villeags,87.5,1,rhondavilleags,92.857143,1
2,ryan,ryan,100.0,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,80.0,3,gonzales,gonzales,100.0,3,chleogonzales,92.307692,3
4,tyler,tyler,100.0,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.0,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.0,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.0,28,clark,clark,100.0,72,jessiacclark,91.666667,75
98,robert,robert,100.0,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


## Optimized RapidFuzz using cdist instead of lambda for each row##

In [84]:
df1 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df100000.csv")
df2 = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df100000.csv")

# Drop rows with NaN values only for specific columns in both dataframes
columns_to_check = ['FirstName', 'LastName']
# df1_cleaned = df1.dropna(subset=columns_to_check)
df2 = df2.dropna(subset=columns_to_check)

print("After dropping rows with NaNs in 'FirstName' and 'LastName':")
# print("df1_cleaned:")
# print(df1_cleaned.isnull().sum())

print("df2 cleaned:")
print(df2.isnull().sum())
print(df2.shape)
hr_df = df1.copy()
client_df = df2.copy()
client_df['Revenue'] = [randint(2000, 500000) for _ in range(df2.shape[0])]
print(f"DF Shapes: hr_df = {hr_df.shape}, client_df = {client_df.shape}")
hr_df['FirstName'] = hr_df['FirstName'].str.lower()
hr_df['LastName'] = hr_df['LastName'].str.lower()
hr_df['FullName'] = hr_df['FirstName']+hr_df['LastName']
# hr_df.head(10)

client_df['FirstName'] = client_df['FirstName'].str.lower()
client_df['LastName'] = client_df['LastName'].str.lower()
client_df['FullName'] = client_df['FirstName']+client_df['LastName']
# client_df.head(10)

After dropping rows with NaNs in 'FirstName' and 'LastName':
df2 cleaned:
Unnamed: 0        0
FirstName         0
LastName          0
DateOfBirth    7994
dtype: int64
(80898, 4)
DF Shapes: hr_df = (100000, 4), client_df = (80898, 5)


,Unnamed: 0,FirstName,LastName,DateOfBirth,Revenue,FullName
0,0,billy,davidson,1997-11-14,58499,billydavidson
1,1,kristen,rsos,2012-10-01,469543,kristenrsos
2,2,anna,york,2010-12-23,475167,annayork
3,3,danielle,alng,1946-06-08,9509,daniellealng
4,4,isaac,ajckson,NaN,486315,isaacajckson
6,6,steven,hunter,1926-04-13,74026,stevenhunter
7,7,terry,hernandez,1999-09-08,73543,terryhernandez
8,8,lauren,evans,1952-02-07,306166,laurenevans
10,10,vanessa,taylor,1964-09-29,323401,vanessataylor
11,11,monica,levy,NaN,197243,monicalevy


In [85]:
# Prepare lists of names/choices from df2 to match against
client_df_first_names = client_df['FirstName'].dropna().unique()
client_df_last_names = client_df['LastName'].dropna().unique()
client_df_full_names = client_df['FullName'].dropna().unique()

# Optimized function to get the best match using cdist
def optimized_best_match(df_col, choices, scorer):
    results = rapidfuzz.process.cdist(queries=df_col.dropna().tolist(), choices=choices, scorer=scorer)
    best_matches = results.argmax(axis=1)
    best_scores = results.max(axis=1)
    matched_values = [choices[idx] for idx in best_matches]

    # Create a DataFrame to hold the results
    matched_df = pd.DataFrame({
        'Matched': matched_values,
        'Score': best_scores,
        'Index': best_matches
    }, index=df_col.dropna().index)

    # Reindex to the original DataFrame's index
    matched_df = matched_df.reindex(df_col.index, fill_value=None)

    return matched_df['Matched'], matched_df['Score'], matched_df['Index']

# Apply the optimized function
def optimized_rapidfuzz_best_match_with_score(param_df, scorer) -> pd.DataFrame:
    df = param_df.copy()
    df['MatchedFullName'], df['FullNameScore'], df['FullNameIndex'] = optimized_best_match(hr_df['FullName'], client_df_full_names, scorer)
    df['MatchedFirstName'], df['FirstNameScore'], df['FirstNameIndex'] = optimized_best_match(hr_df['FirstName'], client_df_first_names, scorer)
    df['MatchedLastName'], df['LastNameScore'], df['LastNameIndex'] = optimized_best_match(hr_df['LastName'], client_df_last_names, scorer)

    # Fill NaN values for rows that were originally NaN
    df['MatchedFullName'].fillna('_', inplace=True)
    df['FullNameScore'].fillna(0, inplace=True)
    df['FullNameIndex'].fillna(-1, inplace=True)
    df['MatchedFirstName'].fillna('_', inplace=True)
    df['FirstNameScore'].fillna(0, inplace=True)
    df['FirstNameIndex'].fillna(-1, inplace=True)
    df['MatchedLastName'].fillna('_', inplace=True)
    df['LastNameScore'].fillna(0, inplace=True)
    df['LastNameIndex'].fillna(-1, inplace=True)

    return df



In [86]:
# SIMPLE RATIO METHOD
start_time = time.time()
df_simple_ratio = optimized_rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.ratio)
print(f"Elapsed time simple ratio: {(time.time() - start_time):.2f} seconds")
df_simple_ratio.to_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\output_dataframes\rapidfuzz_optimized_df10k_simple_ratio.csv")
# Show results
df_simple_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time simple ratio: 779.24 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,billy,billy,100.0,0,davidson,davidson,100.0,0,billydavidson,100.000000,0
1,kristen,kristen,100.0,1,ross,ross,100.0,307,kristenrsos,90.909088,1
2,anna,anna,100.0,2,york,york,100.0,2,annayork,100.000000,2
3,danielle,danielle,100.0,3,lang,lang,100.0,405,daniellealng,91.666664,3
4,isaac,isaac,100.0,4,jackson,jackson,100.0,430,isaacajckson,91.666664,4
...,...,...,...,...,...,...,...,...,...,...,...
95,emma,emma,100.0,72,mccullough,mccullough,100.0,1031,emmamcclulough,92.857140,77
96,dawn,dawn,100.0,73,gardner,gardner,100.0,134,dawngarnder,90.909088,78
97,ethan,ethan,100.0,74,stokes,stokes,100.0,1813,ethanstkoes,90.909088,79
98,jennifer,jennifer,100.0,131,stewart,stewart,100.0,411,jenniferstewart,100.000000,25091


In [64]:
# PARITAL RATIO METHOD
start_time = time.time()
df_partial_ratio = optimized_rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.partial_ratio)
print(f"Elapsed time partial ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_partial_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time partial ratio: 2.11 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.000000,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.000000,1,villegas,le,100.0,359,rhondavilleags,92.857140,1
2,ryan,ryan,100.000000,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,88.888885,3,gonzales,gonzales,100.0,3,chleogonzales,92.307693,3
4,tyler,tyler,100.000000,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.000000,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.000000,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.000000,28,clark,clark,100.0,72,jessiacclark,91.666664,75
98,robert,robert,100.000000,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


In [65]:
# TOKEN SORT RATIO METHOD
start_time = time.time()
df_token_sort_ratio = optimized_rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.token_sort_ratio)
print(f"Elapsed time token sort ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_token_sort_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time token sort ratio: 0.09 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.0,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.0,1,villegas,villeags,87.5,1,rhondavilleags,92.857140,1
2,ryan,ryan,100.0,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,80.0,3,gonzales,gonzales,100.0,3,chleogonzales,92.307693,3
4,tyler,tyler,100.0,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.0,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.0,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.0,28,clark,clark,100.0,72,jessiacclark,91.666664,75
98,robert,robert,100.0,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


In [66]:
# TOKEN SET RATIO METHOD
start_time = time.time()
df_token_set_ratio = optimized_rapidfuzz_best_match_with_score(param_df = hr_df, scorer = rapidfuzz.fuzz.token_set_ratio)
print(f"Elapsed time token set ratio: {(time.time() - start_time):.2f} seconds")
# Show results
df_token_set_ratio[['FirstName', 'MatchedFirstName', 'FirstNameScore', 'FirstNameIndex', 'LastName', 'MatchedLastName','LastNameScore','LastNameIndex','MatchedFullName','FullNameScore','FullNameIndex']].head(100)

Elapsed time token set ratio: 2.85 seconds


,FirstName,MatchedFirstName,FirstNameScore,FirstNameIndex,LastName,MatchedLastName,LastNameScore,LastNameIndex,MatchedFullName,FullNameScore,FullNameIndex
0,sarah,sarah,100.0,0,mitchell,mitchell,100.0,0,sarahmitchell,100.000000,0
1,rhonda,rhonda,100.0,1,villegas,villeags,87.5,1,rhondavilleags,92.857140,1
2,ryan,ryan,100.0,2,jones,jones,100.0,77,ryanjones,100.000000,612
3,chloe,chleo,80.0,3,gonzales,gonzales,100.0,3,chleogonzales,92.307693,3
4,tyler,tyler,100.0,4,serrano,serrano,100.0,4,tylerserrano,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
95,joseph,joseph,100.0,104,leon,leon,100.0,70,jsoephleon,90.000000,73
96,cynthia,cynthia,100.0,67,arroyo,arroyo,100.0,71,cynthiaarroyo,100.000000,74
97,jessica,jessica,100.0,28,clark,clark,100.0,72,jessiacclark,91.666664,75
98,robert,robert,100.0,12,ramirez,ramirez,100.0,73,robertramirez,100.000000,76


In [92]:
client_df.head(10)

,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName
0,0,Billy,Davidson,1997-11-14,Billy Davidson
1,1,Kristen,Rsos,2012-10-01,Kristen Rsos
2,2,Anna,York,2010-12-23,Anna York
3,3,Danielle,aLng,1946-06-08,Danielle aLng
4,4,Isaac,aJckson,NaN,Isaac aJckson
5,5,Michele,NaN,1996-10-26,NaN
6,6,Steven,Hunter,1926-04-13,Steven Hunter
7,7,Terry,Hernandez,1999-09-08,Terry Hernandez
8,8,Lauren,Evans,1952-02-07,Lauren Evans
9,9,Christophre,NaN,1923-10-25,NaN


## Optimized Parallel RapidFuzz using cdist instead of lambda for each row ##

In [91]:
import pandas as pd
import numpy as np
import rapidfuzz
from rapidfuzz import fuzz

from joblib import Parallel, delayed
import multiprocessing

# Ingesting the csv dataframes
hr_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df100000.csv")
client_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df100000.csv")
hr_df['FullName'] = hr_df['FirstName']+' '+hr_df['LastName']
client_df['FullName'] = client_df['FirstName']+' '+client_df['LastName']

# Prepare lists of names/choices from client_df to match against
client_df_full_names = client_df['FullName'].dropna().unique()
client_df_first_names = client_df['FirstName'].dropna().unique()
client_df_last_names = client_df['LastName'].dropna().unique()

def optimized_best_match(df_col, choices, scorer):
    results = rapidfuzz.process.cdist(queries=df_col.dropna().tolist(), choices=choices, scorer=scorer)
    # results = rapidfuzz.process.cdist(queries=df_col.dropna().tolist(), choices=choices, scorer=scorer)

    best_matches = results.argmax(axis=1)
    best_scores = results.max(axis=1)
    matched_values = [choices[idx] for idx in best_matches]

    # Create a DataFrame to hold the results
    matched_df = pd.DataFrame({
        'Matched': matched_values,
        'Score': best_scores,
        'Index': best_matches
    }, index=df_col.dropna().index)

    # Reindex to the original DataFrame's index
    matched_df = matched_df.reindex(df_col.index, fill_value=None)

    return matched_df['Matched'], matched_df['Score'], matched_df['Index']

def parallel_optimized_best_match(df_col, choices, scorer, n_jobs):
    # Split the dataframe column into chunks
    chunk_size = len(df_col) // n_jobs
    chunks = [df_col[i:i + chunk_size] for i in range(0, len(df_col), chunk_size)]

    # Parallel processing using joblib
    results = Parallel(n_jobs=n_jobs)(delayed(optimized_best_match)(chunk, choices, scorer) for chunk in chunks)

    # Concatenate the results
    matched = pd.concat([result[0] for result in results])
    scores = pd.concat([result[1] for result in results])
    indices = pd.concat([result[2] for result in results])

    return matched, scores, indices

# Apply the optimized function
def optimized_rapidfuzz_best_match_with_score(param_df, scorer) -> pd.DataFrame:
    df = param_df.copy()
    n_jobs = multiprocessing.cpu_count()

    df['MatchedFullName'], df['FullNameScore'], df['FullNameIndex'] = parallel_optimized_best_match(df['FullName'], client_df_full_names, scorer, n_jobs)
    df['MatchedFirstName'], df['FirstNameScore'], df['FirstNameIndex'] = parallel_optimized_best_match(df['FirstName'], client_df_first_names, scorer, n_jobs)
    df['MatchedLastName'], df['LastNameScore'], df['LastNameIndex'] = parallel_optimized_best_match(df['LastName'], client_df_last_names, scorer, n_jobs)

    # Fill NaN values for rows that were originally NaN
    df['MatchedFullName'].fillna('_', inplace=True)
    df['FullNameScore'].fillna(0, inplace=True)
    df['FullNameIndex'].fillna(-1, inplace=True)
    df['MatchedFirstName'].fillna('_', inplace=True)
    df['FirstNameScore'].fillna(0, inplace=True)
    df['FirstNameIndex'].fillna(-1, inplace=True)
    df['MatchedLastName'].fillna('_', inplace=True)
    df['LastNameScore'].fillna(0, inplace=True)
    df['LastNameIndex'].fillna(-1, inplace=True)

    return df

# Run the optimized function
scorer = rapidfuzz.fuzz.ratio
hr_df = optimized_rapidfuzz_best_match_with_score(hr_df, scorer)

# Show results
print(hr_df.head())


   Unnamed: 0 FirstName  LastName DateOfBirth        FullName MatchedFullName  \
0           0     Billy  Davidson  1997-11-14  Billy Davidson  Billy Davidson   
1           1   Kristen      Ross  2012-10-01    Kristen Ross    Kristen Rsos   
2           2      Anna      York  2010-12-23       Anna York       Anna York   
3           3  Danielle      Lang  1946-06-08   Danielle Lang   Danielle aLng   
4           4     Isaac   Jackson  1941-07-24   Isaac Jackson   Isaac aJckson   

   FullNameScore  FullNameIndex MatchedFirstName  FirstNameScore  \
0     100.000000              0            Billy           100.0   
1      91.666664              1          Kristen           100.0   
2     100.000000              2             Anna           100.0   
3      92.307693              3         Danielle           100.0   
4      92.307693              4            Isaac           100.0   

   FirstNameIndex MatchedLastName  LastNameScore  LastNameIndex  
0               0        Davidson     

## Test Code to optimize rapidfuzz lambda vs cdist ##

In [26]:
import pandas as pd
from rapidfuzz import process, fuzz

# Example dataframes
# hr_df = pd.DataFrame({
#     'FullName': ['michelle gonzalez', 'john doe', 'jane smith'],
#     'FirstName': ['michelle', 'john', 'jane'],
#     'LastName': ['gonzalez', 'doe', 'smith']
# })
# client_df = pd.DataFrame({
#     'FullName': ['michelle ognzalez', 'john smith', 'jane smth'],
#     'FirstName': ['michelle', 'john', 'jane'],
#     'LastName': ['ognzalez', 'smith', 'smth']
# })
hr_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df1.csv")
client_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df1.csv")
hr_df['FullName'] = hr_df['FirstName']+' '+hr_df['LastName']
client_df['FullName'] = client_df['FirstName']+' '+client_df['LastName']


# Prepare lists of names/choices from client_df to match against
client_df_full_names = client_df['FullName'].dropna().tolist()
client_df_first_names = client_df['FirstName'].dropna().tolist()
client_df_last_names = client_df['LastName'].dropna().tolist()

# Function to find the best match with score and index
def best_match_with_score(row, choices):
    if pd.isnull(row):
        return None, 0, -1
    match = process.extractOne(row, choices, scorer=fuzz.token_sort_ratio)
    if match:
        matched_value, score, match_index = match[0], match[1], choices.index(match[0])
        return matched_value, score, match_index
    return None, 0, -1
start_time = time.time()
# Apply fuzzy matching to hr_df using the best_match_with_score function
hr_df['MatchedFullName'], hr_df['FullNameScore'], hr_df['FullNameIndex'] = zip(*hr_df['FullName'].apply(lambda x: best_match_with_score(x, client_df_full_names)))
hr_df['MatchedFirstName'], hr_df['FirstNameScore'], hr_df['FirstNameIndex'] = zip(*hr_df['FirstName'].apply(lambda x: best_match_with_score(x, client_df_first_names)))
hr_df['MatchedLastName'], hr_df['LastNameScore'], hr_df['LastNameIndex'] = zip(*hr_df['LastName'].apply(lambda x: best_match_with_score(x, client_df_last_names)))
print(f"Spent time for lambda: {(time.time() - start_time):.2f} seconds")

# Fill NaN values for rows that were originally NaN
hr_df['MatchedFullName'].fillna('_', inplace=True)
hr_df['FullNameScore'].fillna(0, inplace=True)
hr_df['FullNameIndex'].fillna(-1, inplace=True)
hr_df['MatchedFirstName'].fillna('_', inplace=True)
hr_df['FirstNameScore'].fillna(0, inplace=True)
hr_df['FirstNameIndex'].fillna(-1, inplace=True)
hr_df['MatchedLastName'].fillna('_', inplace=True)
hr_df['LastNameScore'].fillna(0, inplace=True)
hr_df['LastNameIndex'].fillna(-1, inplace=True)

# Show results
hr_df.head(100)

Spent time for lambda: 0.83 seconds


C:\Users\dalia\AppData\Local\Temp\ipykernel_21600\10941897.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hr_df['MatchedFullName'].fillna('_', inplace=True)
C:\Users\dalia\AppData\Local\Temp\ipykernel_21600\10941897.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName,MatchedFullName,FullNameScore,FullNameIndex,MatchedFirstName,FirstNameScore,FirstNameIndex,MatchedLastName,LastNameScore,LastNameIndex
0,0,Sarah,Mitchell,1954-04-06,Sarah Mitchell,Sarah Mitchell,100.000000,0,Sarah,100.0,0,Mitchell,100.0,0
1,1,Rhonda,Villegas,1957-03-06,Rhonda Villegas,Rhonda Villeags,93.333333,1,Rhonda,100.0,1,Villeags,87.5,1
2,2,Ryan,Jones,1955-11-02,Ryan Jones,Ryan Jones,100.000000,613,Ryan,100.0,2,Jones,100.0,88
3,3,Chloe,Gonzales,1964-11-07,Chloe Gonzales,Chleo Gonzales,92.857143,3,Chleo,80.0,3,Gonzales,100.0,3
4,4,Tyler,Serrano,2014-03-09,Tyler Serrano,Tyler Serrano,100.000000,4,Tyler,100.0,4,Serrano,100.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Joseph,Leon,2019-06-01,Joseph Leon,Jsoeph Leon,90.909091,73,Joseph,100.0,139,Leon,100.0,81
96,96,Cynthia,Arroyo,1937-05-07,Cynthia Arroyo,Cynthia Arroyo,100.000000,74,Cynthia,100.0,86,Arroyo,100.0,82
97,97,Jessica,Clark,1928-03-25,Jessica Clark,Jessiac Clark,92.307692,75,Jessica,100.0,34,Clark,100.0,83
98,98,Robert,Ramirez,1993-05-11,Robert Ramirez,Robert Ramirez,100.000000,76,Robert,100.0,12,Ramirez,100.0,84


In [27]:
import pandas as pd
from rapidfuzz import process, fuzz
import numpy as np

# Example dataframes
# hr_df = pd.DataFrame({
#     'FullName': ['michelle gonzalez', 'john doe', 'jane smith'],
#     'FirstName': ['michelle', 'john', 'jane'],
#     'LastName': ['gonzalez', 'doe', 'smith']
# })

# client_df = pd.DataFrame({
#     'FullName': ['michelle ognzalez', 'john smith', 'jane smth'],
#     'FirstName': ['michelle', 'john', 'jane'],
#     'LastName': ['ognzalez', 'smith', 'smth']
# })
hr_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\hr_df1.csv")
client_df = pd.read_csv(r"C:\Users\dalia\Desktop\Radu\DeutscheBank Fuzzy Matching\dataframes\client_df1.csv")
hr_df['FullName'] = hr_df['FirstName']+' '+hr_df['LastName']
client_df['FullName'] = client_df['FirstName']+' '+client_df['LastName']

# Prepare lists of names/choices from client_df to match against
client_df_full_names = client_df['FullName'].dropna().tolist()
client_df_first_names = client_df['FirstName'].dropna().tolist()
client_df_last_names = client_df['LastName'].dropna().tolist()

# Optimized function to get the best match using cdist
def optimized_best_match(df_col, choices):
    results = process.cdist(df_col.dropna().tolist(), choices, scorer=fuzz.token_sort_ratio)
    best_matches = results.argmax(axis=1)
    best_scores = results.max(axis=1)
    matched_values = [choices[idx] for idx in best_matches]

    # Create a DataFrame to hold the results
    matched_df = pd.DataFrame({
        'Matched': matched_values,
        'Score': best_scores,
        'Index': best_matches
    }, index=df_col.dropna().index)

    # Reindex to the original DataFrame's index
    matched_df = matched_df.reindex(df_col.index, fill_value=None)

    return matched_df['Matched'], matched_df['Score'], matched_df['Index']

# Apply the optimized function
start_time = time.time()
hr_df['MatchedFullName'], hr_df['FullNameScore'], hr_df['FullNameIndex'] = optimized_best_match(hr_df['FullName'], client_df_full_names)
hr_df['MatchedFirstName'], hr_df['FirstNameScore'], hr_df['FirstNameIndex'] = optimized_best_match(hr_df['FirstName'], client_df_first_names)
hr_df['MatchedLastName'], hr_df['LastNameScore'], hr_df['LastNameIndex'] = optimized_best_match(hr_df['LastName'], client_df_last_names)
print(f"Spent time for lambda: {(time.time() - start_time):.2f} seconds")

# Fill NaN values for rows that were originally NaN
hr_df['MatchedFullName'].fillna('_', inplace=True)
hr_df['FullNameScore'].fillna(0, inplace=True)
hr_df['FullNameIndex'].fillna(-1, inplace=True)
hr_df['MatchedFirstName'].fillna('_', inplace=True)
hr_df['FirstNameScore'].fillna(0, inplace=True)
hr_df['FirstNameIndex'].fillna(-1, inplace=True)
hr_df['MatchedLastName'].fillna('_', inplace=True)
hr_df['LastNameScore'].fillna(0, inplace=True)
hr_df['LastNameIndex'].fillna(-1, inplace=True)

# Show results
hr_df.head(100)

Spent time for lambda: 0.26 seconds


C:\Users\dalia\AppData\Local\Temp\ipykernel_21600\2644973865.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hr_df['MatchedFullName'].fillna('_', inplace=True)
C:\Users\dalia\AppData\Local\Temp\ipykernel_21600\2644973865.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

,Unnamed: 0,FirstName,LastName,DateOfBirth,FullName,MatchedFullName,FullNameScore,FullNameIndex,MatchedFirstName,FirstNameScore,FirstNameIndex,MatchedLastName,LastNameScore,LastNameIndex
0,0,Sarah,Mitchell,1954-04-06,Sarah Mitchell,Sarah Mitchell,100.000000,0,Sarah,100.0,0,Mitchell,100.0,0
1,1,Rhonda,Villegas,1957-03-06,Rhonda Villegas,Rhonda Villeags,93.333336,1,Rhonda,100.0,1,Villeags,87.5,1
2,2,Ryan,Jones,1955-11-02,Ryan Jones,Ryan Jones,100.000000,613,Ryan,100.0,2,Jones,100.0,88
3,3,Chloe,Gonzales,1964-11-07,Chloe Gonzales,Chleo Gonzales,92.857140,3,Chleo,80.0,3,Gonzales,100.0,3
4,4,Tyler,Serrano,2014-03-09,Tyler Serrano,Tyler Serrano,100.000000,4,Tyler,100.0,4,Serrano,100.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Joseph,Leon,2019-06-01,Joseph Leon,Jsoeph Leon,90.909088,73,Joseph,100.0,139,Leon,100.0,81
96,96,Cynthia,Arroyo,1937-05-07,Cynthia Arroyo,Cynthia Arroyo,100.000000,74,Cynthia,100.0,86,Arroyo,100.0,82
97,97,Jessica,Clark,1928-03-25,Jessica Clark,Jessiac Clark,92.307693,75,Jessica,100.0,34,Clark,100.0,83
98,98,Robert,Ramirez,1993-05-11,Robert Ramirez,Robert Ramirez,100.000000,76,Robert,100.0,12,Ramirez,100.0,84
